In [1]:
import os
os.chdir("../")

In [2]:
import requests

# Your modified OneDrive direct download link
url = 'https://brpucrs-my.sharepoint.com/:u:/g/personal/guido_mainardi_edu_pucrs_br/EV87RzSfkVBMhcO0LmxXD8cBgwdMy6EzDwBspjLEi24BRQ?e=a2fX8n'

# Local path to save the downloaded file
file_path = 'your_file.zip'

# GET request to download the file
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Write the content to a file
with open(file_path, 'wb') as f:
    f.write(response.content)

print('Download completed!')


/Users/guido/Library/CloudStorage/OneDrive-PUCRS-BR/Mestrado/Dissertacao/Howler-Monkey/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Download completed!


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    data_id: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from HowlerMonkey.constants import *
from HowlerMonkey.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir        = config.root_dir,
            data_id         = config.data_id,
            local_data_file = config.local_data_file,
            unzip_dir       = config.unzip_dir 
        )
        
        
        return data_ingestion_config

In [30]:
import os
import zipfile
import gdown
from HowlerMonkey import logger
from HowlerMonkey.utils.common import get_size

In [31]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_id = self.config.data_id
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data with id {dataset_id} into file {zip_download_dir}")

            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(
                prefix+dataset_id,
                zip_download_dir
            )

            logger.info(f"Downloaded data with id {dataset_id} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [32]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-28 22:47:48,124: INFO: common] yaml file: config/config.yaml loaded successfully
[2024-03-28 22:47:48,126: INFO: common] yaml file: params.yaml loaded successfully
[2024-03-28 22:47:48,127: INFO: common] Creating directory: artifacts
[2024-03-28 22:47:48,128: INFO: common] Creating directory: artifacts/data_ingestion
[2024-03-28 22:47:48,128: INFO: 1690253806] Downloading data with id 1mIR2ocvXOn2eIVnEUGuaxW5ta3fLoVsx into file artifacts/data_ingestion/howler_data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1mIR2ocvXOn2eIVnEUGuaxW5ta3fLoVsx
From (redirected): https://drive.google.com/uc?/export=download&id=1mIR2ocvXOn2eIVnEUGuaxW5ta3fLoVsx&confirm=t&uuid=58724ec2-11c0-47ae-9b37-f6692ad937bf
To: /Users/guido/Library/CloudStorage/OneDrive-PUCRS-BR/Mestrado/Dissertacao/Howler-Monkey/artifacts/data_ingestion/howler_data.zip
 36%|███▌      | 553M/1.55G [05:31<07:12, 2.31MB/s] 

KeyboardInterrupt: 

 36%|███▌      | 553M/1.55G [05:49<07:12, 2.31MB/s]